In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from glob import glob

import numpy as np
import pandas as pd

In [ ]:
from xray import data, trainer, utils

In [ ]:
path_to_csv = "../../raw_data/sample-data/"
csv_file = "sample_labels.csv"

In [ ]:
df = data.get_data(os.path.join(path_to_csv, csv_file))

In [ ]:
df.drop(
    columns=[
        "Follow-up #",
        "Patient Age",
        "Patient Gender",
        "View Position",
        "OriginalImagePixelSpacing_x",
        "OriginalImagePixelSpacing_y",
        "OriginalImageWidth",
        "OriginalImageHeight",
    ],
    inplace=True,
    errors="ignore",
)

In [ ]:
utils.get_paths(df, path_to_csv, verbose=0)

In [ ]:
df["Fixed_Labels"] = df["Finding Labels"].map(lambda x: x.split("|"))
df.head(3)

In [ ]:
# Keep relative paths
df.path = df.path.map(lambda x: "/".join(x.split("/")[-3:]))

In [ ]:
df_sick = df[df["Finding Labels"] != "No Finding"]

In [ ]:
ds_train, ds_val, ds_test = data.split_df(
    dataset=df_sick,
    column_to_filter_by="Patient ID",
    train_val_test=(0.65, 0.15, 0.15),
    total_filter=1,
)
print("train :", ds_train.shape)
print("val :", ds_val.shape)
print("test :", ds_test.shape)

In [ ]:
input_shape = (224, 224)

In [ ]:
gen_train = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_train,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=32,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=False,
    data_augment=False,
)

gen_val = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_val,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=32,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=False,
    data_augment=False,
)

In [ ]:
gen_test = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_test,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=32,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=True,
    data_augment=False,
)


In [ ]:
total_classes = len(model.gen_train.class_indices)
total_classes

In [ ]:
model = trainer.Trainer(
    gen_train=gen_train,
    gen_val=gen_val,
    category_type="multilabel",
)

In [ ]:
cnn_arch = {
    "input_shape": input_shape,
    "output_shape": total_classes,
    "output_activation": "sigmoid",
    "dense_layer_geometry": (1024, 512, 256),
    'transfer_model': 'VGG16',
    "dense_layer_activation": "relu",
    "dropout_layers": False,
}

In [ ]:
model.build_cnn(**cnn_arch)

In [ ]:
model.compile_model()

In [ ]:
model.experiment_name

In [ ]:
history = model.fit_model(epochs=1)

In [ ]:
history.history

In [ ]:
evalu = model.pipeline.evaluate(gen_test, workers=4, use_multiprocessing=True)

In [ ]:
evalu

In [ ]:
model.pipeline.

In [ ]:
model.pipeline.compiled_metrics.metrics

In [ ]:
model.pipeline.compiled_loss.metrics.

In [ ]:
from tensorflow import image